# Tarea 2 - Pregunta 1

## Implementación Merkle-Damgard con Davies-Meyer y padding

*Nota*: Este archivo fue probado utilizando Google Colab.

In [3]:
from typing import Callable
from sys import byteorder

# Obtenido de https://nitratine.net/blog/post/xor-python-byte-strings/
def byte_xor(ba1: bytearray, ba2: bytearray) -> bytearray:
    return bytearray([a ^ b for a, b in zip(ba1, ba2)])

def davies_meyer(encrypt: Callable[[bytearray, bytearray],bytearray], l_key: int, l_message: int) -> Callable[[bytearray], bytearray]:
    """
    Arguments :
    encrypt : an encryption function
    l_key : length in bytes of the keys for encrypt
    l_message : length in bytes of the messages for encrypt
    Returns :
    A compression function from messages of length l_key + l_message to
    messages of length l_message , defined by using the Davies - Meyer
    construction
    """
    def compression(message: bytearray) -> bytearray:
        u = message[:l_key]
        v = message[l_key: l_message + l_key]
        return byte_xor(encrypt(u, v), v)

    return compression

def pad(message: bytearray, l_block: int) -> bytearray:
    """
    Arguments :
    message : message to be padded
    l_block : length in bytes of the block
    Returns :
    extension of message that includes the length of message
    ( in bytes ) in its last block
    """
    padded = bytearray(message[:])
    l_m = len(padded)
    # Bloque final con el largo del mensaje
    last_block = l_m.to_bytes(l_block, byteorder='big')
    if (len(message) % l_block == 0):
        padded.extend(last_block)
        return padded
    # Se obtiene el bloque incompleto a rellenar y se rellena con 100000...
    l_block_to_fill = l_m % l_block
    block_to_fill = padded[-l_block_to_fill:]
    l_to_fill = l_block - l_block_to_fill
    padded.extend(bytearray(b'\x01') + bytearray(l_to_fill - 1))
    # Se anade el ultimo bloque
    padded.extend(last_block)
    return padded

def merkle_damgard ( IV : bytearray , comp : Callable[[bytearray], bytearray] ,l_block : int ) -> Callable[[bytearray], bytearray]:
    """
    Arguments :
    IV : initialization vector for a hash function
    comp : compression function to be used in the Merkle - Damgard construction
    l_block : length in bytes of the blocks to be used in the Merkle - Damgard construction
    Returns :
    A hash function for messages of arbitrary length , defined by using the Merkle - Damgard construction
    """
    def hash(message: bytearray):
        msg = pad(message, l_block)
        h_i = IV
        for i in range(len(msg)//l_block):
            block = msg[l_block * i: l_block * i + l_block]
            h_i = comp(block + h_i)
        return h_i

    return hash

## Test de la solución
En las siguientes celdas se realiza el test de prueba entregado en la issues del repositorio del curso.

Primero, se deben instalar las dependencias.

In [4]:
!pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 13.8 MB/s 


In [14]:
from Crypto.Cipher import AES

def encrypt(key: bytearray, msg: bytearray) -> bytearray:
    alg = AES.new(key, AES.MODE_ECB)
    return alg.encrypt(msg)

def AES_128(key: bytearray, message: bytearray) -> bytearray:
    a = AES.new(key, AES.MODE_ECB)
    return bytearray(a.encrypt(message))

if __name__ == "__main__":
    compresion = davies_meyer(AES_128, 16, 16)
    hash = merkle_damgard(bytearray(b'1234567890123456'), compresion, 16)

    s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
    s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
    s3 = bytearray(b'Un mensaje corto')
    s4 = bytearray(b'')

    h1 = hash(s1)
    h2 = hash(s2)
    h3 = hash(s3)
    h4 = hash(s4)
    
    print("[-------------------------------- Outputs obtenidos -----------------------------------]")
    print(h1)
    print(h2)
    print(h3)
    print(h4)
    print()
    print("[-------------------------------- Verificación del output -----------------------------------]")
    print(h1 == bytearray(b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea'))
    print(h2 == bytearray(b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96'))
    print(h3 == bytearray(b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81'))
    print(h4 == bytearray(b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc'))

[-------------------------------- Outputs obtenidos -----------------------------------]
bytearray(b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea')
bytearray(b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96')
bytearray(b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81')
bytearray(b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc')

[-------------------------------- Verificación del output -----------------------------------]
True
True
True
True
